# 0.0 Config

In [18]:
 ! huggingface-cli login --token 'your-token'
    
# from huggingface_hub import login
# login() #token



The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!pip install tensorboard
!pip install huggingface_hub
%pip install ultralytics
import ultralytics
ultralytics.checks()
import torch
torch.cuda.empty_cache()

# Train

In [ ]:
# import libraries
from ultralytics import YOLO
import os

# Paths to dataset and configuration file
dataset_path = "/content/datasets/DOTAv1.5"
# data_yaml = "/content/datasets/DOTAv1.5.yaml"

# Check if the dataset exists
if not os.path.exists(dataset_path):
    print(f"Dataset not found at path {dataset_path}.")
else:
    print("Dataset is ready.")

# Load the YOLOv8-OBB model
model = YOLO("/content/yolo11n-obb.pt")  # Pre-trained YOLOv8 model with OBB support

# Training settings
epochs = 150
img_size = 640
batch_size = 32  # Set according to GPU memory
learning_rate = 0.001  # Initial learning rate

# Train the model
results = model.train(
    data='DOTAv1.5.yaml',              # Dataset configuration file
    epochs=epochs,                     # Total number of epochs
    imgsz=img_size,                    # Image size
    batch=batch_size,                  # Batch size
    lr0=learning_rate,                 # Initial learning rate
    optimizer='Adam',                  # Adam optimizer
    project="runs/train",              # Project path for saving logs
    name="yolo_aerial_mine_detection",  # Project name
    val=True,                          # Enable validation each epoch
    workers=4,                         # Number of threads for data loading
    device='0,1'
)

# Evaluate the model after training
metrics = model.val(data='DOTAv1.5.yaml')

# Print evaluation metrics
print(f"mAP@0.5: {metrics.box.map50:.3f}")
print(f"mAP@[0.5:0.95]: {metrics.box.map:.3f}")
print(f"Precision: {metrics.box.precision:.3f}")
print(f"Recall: {metrics.box.recall:.3f}")


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [15]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))
download_file('/kaggle/working/runs', 'out')


/kaggle/working/out.zip

In [24]:
from huggingface_hub import create_repo, upload_folder

# ایجاد یک مخزن جدید
repo_name = "repo-name"
token='toekn'
repo_url = create_repo(repo_name, token=token)

# load folder in to Hugging Face Hub
upload_folder(
    folder_path="/kaggle/working/runs/train/yolo_aerial_mine_detection",  # مسیر دایرکتوری حاوی فایل‌های مدل
    repo_id=repo_name,
    token=token
)


Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1730103318.23b36f4d79a7.318.0:   0%|          | 0.00/369k [00:00<?, ?B/s]

best.pt:   0%|          | 0.00/5.70M [00:00<?, ?B/s]

last.pt:   0%|          | 0.00/5.70M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Ashegh-Sad-Warrior/yolo_aerial_detection_/commit/cd839aa4bd4924db3147652a7869c746bee3f29b', commit_message='Upload folder using huggingface_hub', commit_description='', oid='cd839aa4bd4924db3147652a7869c746bee3f29b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Ashegh-Sad-Warrior/yolo_aerial_detection_', endpoint='https://huggingface.co', repo_type='model', repo_id='Ashegh-Sad-Warrior/yolo_aerial_detection_'), pr_revision=None, pr_num=None)

In [ ]:
# ارزیابی مدل پس از آموزش
metrics = model.val(data='DOTAv1.5.yaml')

# چاپ معیارهای ارزیابی
print(f"mAP@0.5: {metrics.box.map50:.3f}")
print(f"mAP@[0.5:0.95]: {metrics.box.map:.3f}")
print(f"Precision: {metrics.box.mp():.3f}")
print(f"Recall: {metrics.box.mr():.3f}")

